In [0]:
import requests, base64, io

# GitHub folder API endpoint
repo_api = "https://api.github.com/repos/pradeep-1004/git_API_test_dataset/contents/"

bronze_base = "/Volumes/parent_cata/git_pull/api_pull"


# 1. Get list of all files in repo
response = requests.get(repo_api)
files_metadata = response.json()

for file in files_metadata:
    
    # Skip non-CSV files
    if not file["name"].lower().endswith(".csv"):
        print("Skipping:", file["name"])
        continue
    
    print("Processing:", file["name"])
    
    # 2. Download file content from GitHub API
    file_response = requests.get(file["download_url"])
    csv_text = file_response.text

    # 3. Convert to Spark DataFrame
    df = (
        spark.read
            .option("header", True)
            .option("inferSchema", True)
            .csv(io.StringIO(csv_text))
    )

    # 4. Save to Delta (Bronze)
    target = f"{bronze_base}/{file['name'].replace('.csv','')}"
    df.write.format("delta").mode("overwrite").save(target)
    
    print("Saved to:", target)

print("✔ All CSV files ingested automatically!")


In [0]:
import requests
import pandas as pd

repo_api = "https://api.github.com/repos/pradeep-1004/git_API_test_dataset/contents/"
bronze_base = "/Volumes/parent_cata/git_pull/api_pull"

response = requests.get(repo_api)
files_metadata = response.json()

for file in files_metadata:
    if not file["name"].lower().endswith(".csv"):
        print("Skipping:", file["name"])
        continue

    print("Processing:", file["name"])
    file_response = requests.get(file["download_url"])
    csv_text = file_response.text

    # Read CSV into pandas DataFrame
    pdf = pd.read_csv(io.StringIO(csv_text))

    # Convert pandas DataFrame to Spark DataFrame
    df = spark.createDataFrame(pdf)

    target = f"{bronze_base}/{file['name'].replace('.csv','')}"
    df.write.format("delta").mode("overwrite").save(target)
    print("Saved to:", target)

print("✔ All CSV files ingested automatically!")